# Stage 2 | Simple api methods

In [1]:
import requests as r


class Elastic:
    """
    The so simple wrapper for Elasticsearch.
    """
    headers = {
        'Content-Type': 'application/json'
    }
    url = 'http://elsatic:9200/{path}?pretty'
    
    def get(self, path: str = '/') -> None:
        res =  r.get(self.url.format(path=path))
        print(res.text)
    
    def post(self, path: str = '/', **kwargs) -> None:
        res =  r.post(
            self.url.format(path=path),
            headers=self.headers,
            **kwargs
        )
        print(res.text)
    
    def put(self, path: str = '/', **kwargs) -> None:
        res =  r.put(
            self.url.format(path=path),
            headers=self.headers,
            **kwargs,
        )
        print(res.text)
    
    
    def delete(self, path: str = '/') -> None:
        res =  r.delete(self.url.format(path=path))
        print(res.text)


elastic = Elastic()

elastic.get('/')

{
  "name" : "5TSIKPT",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "_IO8aZNEQh6BjWPJVRvOvQ",
  "version" : {
    "number" : "5.6.4",
    "build_hash" : "8bbedf5",
    "build_date" : "2017-10-31T18:55:38.105Z",
    "build_snapshot" : false,
    "lucene_version" : "6.6.1"
  },
  "tagline" : "You Know, for Search"
}



### Index

In [3]:
# settings

INDEX_NAME = 'stage-2'
TYPE_NAME = 'book'

In [4]:
# create index


elastic.put(f'{INDEX_NAME}')

{
  "error" : {
    "root_cause" : [
      {
        "type" : "index_already_exists_exception",
        "reason" : "index [stage-2/OU4GnLzGQ7-xOpMJkEfRvg] already exists",
        "index_uuid" : "OU4GnLzGQ7-xOpMJkEfRvg",
        "index" : "stage-2"
      }
    ],
    "type" : "index_already_exists_exception",
    "reason" : "index [stage-2/OU4GnLzGQ7-xOpMJkEfRvg] already exists",
    "index_uuid" : "OU4GnLzGQ7-xOpMJkEfRvg",
    "index" : "stage-2"
  },
  "status" : 400
}



In [10]:
# create a first instance
# create auto mapping with instance, fields, version, id
data = {
    "title": "My favourite book",
    "body": "The so boring story."
}

elastic.post(f'{INDEX_NAME}/{TYPE_NAME}/1', json=data)

{
  "_index" : "stage-2",
  "_type" : "book",
  "_id" : "1",
  "_version" : 4,
  "result" : "updated",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "created" : false
}



### Get

In [7]:
# get information

elastic.get(f'{INDEX_NAME}/{TYPE_NAME}/1')

{
  "_index" : "stage-2",
  "_type" : "book",
  "_id" : "1",
  "_version" : 1,
  "found" : true,
  "_source" : {
    "title" : "My favourite book",
    "body" : "The so boring story."
  }
}



### MGet

In [11]:
# false

data = {
    "ids": ["1", "AWSvdbwJKhyi0n1CdVQm"]
}

elastic.post(f'{INDEX_NAME}/{TYPE_NAME}/_mget', json=data)

{
  "docs" : [
    {
      "_index" : "stage-2",
      "_type" : "book",
      "_id" : "1",
      "_version" : 4,
      "found" : true,
      "_source" : {
        "title" : "My favourite book",
        "body" : "The so boring story."
      }
    },
    {
      "_index" : "stage-2",
      "_type" : "book",
      "_id" : "AWSvdbwJKhyi0n1CdVQm",
      "found" : false
    }
  ]
}



### Update

In [12]:
# update query

data = {
    "title": "Theory of elasticsearch!"
}

elastic.put(f'{INDEX_NAME}/{TYPE_NAME}/1', json=data)
elastic.get(f'{INDEX_NAME}/{TYPE_NAME}/1')

{
  "_index" : "stage-2",
  "_type" : "book",
  "_id" : "1",
  "_version" : 5,
  "result" : "updated",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  },
  "created" : false
}

{
  "_index" : "stage-2",
  "_type" : "book",
  "_id" : "1",
  "_version" : 5,
  "found" : true,
  "_source" : {
    "title" : "Theory of elasticsearch!"
  }
}



In [13]:
# revrite

data = {
   "doc": {
      "body": "The so boring story."
   }
}

elastic.post(f'{INDEX_NAME}/{TYPE_NAME}/1/_update', json=data)
elastic.get(f'{INDEX_NAME}/{TYPE_NAME}/1')

{
  "_index" : "stage-2",
  "_type" : "book",
  "_id" : "1",
  "_version" : 6,
  "result" : "updated",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  }
}

{
  "_index" : "stage-2",
  "_type" : "book",
  "_id" : "1",
  "_version" : 6,
  "found" : true,
  "_source" : {
    "title" : "Theory of elasticsearch!",
    "body" : "The so boring story."
  }
}



In [14]:
data = {
    "script" : {
        "source": "ctx._source.views = 0",
        "lang": "painless",
    }
}

elastic.post(f'{INDEX_NAME}/{TYPE_NAME}/1/_update', json=data)
elastic.get(f'{INDEX_NAME}/{TYPE_NAME}/1')

{
  "_index" : "stage-2",
  "_type" : "book",
  "_id" : "1",
  "_version" : 7,
  "result" : "updated",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  }
}

{
  "_index" : "stage-2",
  "_type" : "book",
  "_id" : "1",
  "_version" : 7,
  "found" : true,
  "_source" : {
    "title" : "Theory of elasticsearch!",
    "body" : "The so boring story.",
    "views" : 0
  }
}



In [15]:
data = {
    "script" : {
        "source": "ctx._source.views += 1",
        "lang": "painless",
    }
}

elastic.post(f'{INDEX_NAME}/{TYPE_NAME}/1/_update', json=data)
elastic.get(f'{INDEX_NAME}/{TYPE_NAME}/1')

{
  "_index" : "stage-2",
  "_type" : "book",
  "_id" : "1",
  "_version" : 8,
  "result" : "updated",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  }
}

{
  "_index" : "stage-2",
  "_type" : "book",
  "_id" : "1",
  "_version" : 8,
  "found" : true,
  "_source" : {
    "title" : "Theory of elasticsearch!",
    "body" : "The so boring story.",
    "views" : 1
  }
}



### Update by query

In [ ]:
# _delete_by_query

data = {
    "query": {
      "term": {
        "title": "Theory of elasticsearch!"
      }
    },
    "script" : {
        "source": "ctx._source.body = 'Theory of elasticsearch!'",
        "lang": "painless",
    }
}

elastic.post(f'{INDEX_NAME}/{TYPE_NAME}/_update_by_query', json=data)
elastic.get(f'{INDEX_NAME}/{TYPE_NAME}/1')

### Delete

In [16]:
# delete

elastic.delete(f'{INDEX_NAME}/{TYPE_NAME}/1/')

{
  "found" : true,
  "_index" : "stage-2",
  "_type" : "book",
  "_id" : "1",
  "_version" : 9,
  "result" : "deleted",
  "_shards" : {
    "total" : 2,
    "successful" : 1,
    "failed" : 0
  }
}



In [ ]:
# delete by query

data = {
    "query": {
        "match": {
            "title": "some text it is very some text"
        }
    }
}

elastic.post(f'{INDEX_NAME}/{TYPE_NAME}/_delete_by_query', json=data)

In [ ]:
# Bulk _bulk
data = [
    {"create": {"_id": 1, "_index": INDEX_NAME, "_type": TYPE_NAME, "title": "this is 5 items"}},
    {"create": {"_id": 2, "_index": INDEX_NAME, "_type": TYPE_NAME, "title": "this is 5 items"}},
    {"update": {"_id": 2, "_index": INDEX_NAME, "_type": TYPE_NAME, "description": "Update description"}},
]
